In [65]:

# The %... is an iPython thing, and is not part of the Python language.
# In this case we're just telling the plotting library to draw things on
# the notebook, instead of on a separate window.
%matplotlib inline
# See all the "as ..." contructs? They're just aliasing the package names.
# That way we can call methods like plt.plot() instead of matplotlib.pyplot.plot().
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
import urllib2
import re
from subprocess import PIPE, Popen
import os

In [79]:
geo_noncancer_list = []
other_noncancer_list = []
#read the non-cancer dataset list from csv file
df=pd.read_csv('paper_material/dna-methylation-non-cancer.csv')
#extract the geo identifiers
for i in df.Availability : 
    if re.search("GSE",i):
        geo_noncancer_list.append(i.replace(" ",""))
    else:
        other_noncancer_list.append(i.replace(" ",""))
geo_noncancer_list = list(set(geo_noncancer_list))
print len(geo_noncancer_list)

51


In [80]:
#ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE40nnn/GSE40700/matrix/GSE40700_series_matrix.txt.gz
def check_urllink(ftplink):
    #check if the ftp link exists and check for edge cases 
    try:
        urllib2.urlopen(ftplink)
        return ftplink
    except urllib2.HTTPError, e:
        print(e.code)
        return 1
    except urllib2.URLError, e:
        print(e.args)
        return 1
    
#iterate over the dataset list and fetch files.
def fetch_datasets(geo_noncancer_list):
    gse_noncancer_edge=[]
    failed_fetch_list = []
    for i in geo_noncancer_list:
        key_num = i.split("GSE")[1]
        url_key= "GSE"+key_num[0:2]
        file_n = i + "_series_matrix.txt.gz"
        url = "ftp://ftp.ncbi.nlm.nih.gov/geo/series/"+url_key.replace(" ","")+ "nnn/" + i + "/matrix/" 
        ftplink= (url + file_n).replace(" ","")
        result = check_urllink(ftplink)
        if result == 1:
            gse_noncancer_edge.append(i)
        else:
            print i, " success"
            '''pfetch = Popen("ftp " + ftplink,stdout=PIPE,stderr=PIPE,shell=True,close_fds=True)
            std_out, std_err = pfetch.communicate()
            exit_code = pfetch.returncode
            if exit_code:
                failed_fetch_list.append(i)'''
            #os.system("ftp " + ftplink)
    return gse_noncancer_edge, failed_fetch_list        

In [81]:
gse_noncancer_edge, failed_fetch_list = fetch_datasets(geo_noncancer_list)

GSE28746  success
GSE40279  success
GSE38873  success
GSE27317  success
GSE41826  success
GSE36064  success
('ftp error: [Errno ftp error] 550 GSE15745_series_matrix.txt.gz: No such file or directory',)
GSE37988  success
GSE19711  success
GSE20242  success
GSE32149  success
GSE26974  success
GSE32146  success
GSE38291  success
GSE41782  success
GSE40700  success
GSE37008  success
GSE30601  success
GSE41037  success
GSE30870  success
GSE43269  success
GSE38608  success
GSE34639  success
GSE27097  success
GSE26126  success
GSE30653  success
GSE20236  success
GSE36642  success
GSE37066  success
GSE22595  success
GSE42865  success
GSE35069  success
GSE47627  success
GSE42861  success
GSE36812  success
GSE36166  success
GSE34257  success
GSE34035  success
GSE31848  success
GSE25892  success
GSE42700  success
GSE42510  success
GSE26033  success
GSE30456  success
GSE41169  success
GSE32393  success
GSE30758  success
GSE20067  success
GSE17448  success
GSE44667  success
GSE30090  success


In [71]:
print "number of GEO non-cancer edge cases", len(gse_noncancer_edge)
print "number of other non-cancer sets", len(other_noncancer_list)
print "number of failed fetches for normal GEO data links" , len(failed_fetch_list)


number of GEO non-cancer edge cases 1
number of other non-cancer sets 19
number of failed fetches for normal GEO data links 0


In [166]:
import glob
import gzip
from itertools import islice

def create_methyldf(file1):
    #approx number of lines to fetch tissue information
    N = 2000
    maindf=pd.read_csv(file1,comment="!",sep="\t",index_col='ID_REF')
    f = gzip.open(file1, 'rb')
    #Reading first 2000 lines, header lines are inconsistent but cannot contain such large number. 
    headerlines = islice(f, N)
    for i in headerlines:
        if re.search('!Sample_source_name_ch1',i):
            #tissue_arr = i.replace("!Sample_source_name_ch1",'').split("\t")
            tissue_arr = i.split("\t")
            tissuedf = pd.DataFrame({'tissue_src':tissue_arr[1:]})
            break
    return maindf, tissuedf

In [177]:
datasets = glob.glob("data/geo-noncancer/*txt.gz")
#for now commenting this section out, we will only test one file
'''for file1 in datasets:
    maindf, tissuedf = create_methydf(file1)'''
file1 = 'data/geo-noncancer/GSE20067_series_matrix.txt.gz'
#file1 = 'data/geo-noncancer/GSE15745-GPL8490_series_matrix.txt.gz'
maindf, tissuedf = create_methyldf(file1)

we need to transpose maindf and concatenate with tissuedf to have all the tissue data associated with samples in one place.  Easy for group by operations. For now, just printing maindf and tissuedf below.

In [176]:
maindf

,GSM501487,GSM501488,GSM501489,GSM501490,GSM501491,GSM501492,GSM501493,GSM501494,GSM501495,GSM501496,GSM501497,GSM501498,GSM501499,GSM501500,GSM501501,GSM501502,GSM501503,GSM501504,GSM501505,GSM501506,GSM501507,GSM501508,GSM501509,GSM501510,GSM501511,GSM501512,GSM501513,GSM501514,GSM501515,GSM501516,GSM501517,GSM501518,GSM501519,GSM501520,GSM501521,GSM501522,GSM501523,GSM501524,GSM501525,GSM501526,GSM501527,GSM501528,GSM501529,GSM501530,GSM501531,GSM501532,GSM501533,GSM501534,GSM501535,GSM501536,...,GSM501632,GSM501633,GSM501634,GSM501635,GSM501636,GSM501637,GSM501638,GSM501639,GSM501640,GSM501641,GSM501642,GSM501643,GSM501644,GSM501645,GSM501646,GSM501647,GSM501648,GSM501649,GSM501650,GSM501651,GSM501652,GSM501653,GSM501654,GSM501655,GSM501656,GSM501657,GSM501658,GSM501659,GSM501660,GSM501661,GSM501662,GSM501663,GSM501664,GSM501665,GSM501666,GSM501667,GSM501668,GSM501669,GSM501670,GSM501671,GSM501672,GSM501673,GSM501674,GSM501675,GSM501676,GSM501677,GSM501678,GSM501679,GSM501680,GSM501681
ID_REF,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
cg00000292,0.7450658,0.6817859,0.7086142,0.850587,0.894222,0.8384137,0.7854356,0.817623,0.8269932,0.8282334,0.8093318,0.8289963,0.8456739,0.8053415,0.8522336,0.8036212,0.819,0.8091733,0.8130969,0.8168403,0.7903646,0.7294981,0.8586541,0.8595303,0.8641366,0.7496689,0.8380737,0.7861302,0.7248227,0.7707404,0.7401157,0.8257223,0.7671681,0.8584147,0.8219367,0.8143022,0.75,0.8176665,0.8270572,0.8071842,0.8077446,0.809795,0.816152,0.7286968,0.8259136,0.8573334,0.7468737,0.8132734,0.7831825,0.8465495,...,0.8213097,0.7963875,0.7621998,0.7332293,0.8109366,0.8014842,0.8305476,0.7955233,0.7799876,0.7092449,0.7635025,0.7729694,0.7755539,0.7010551,0.6783349,0.6726886,0.7894737,0.7633726,0.3298969,0.7957983,0.7968606,0.6328012,0.5617021,0.7296768,0.7348485,0.6497396,0.7377819,0.8277512,0.8469914,0.7077315,0.7916386,0.7919109,0.7793765,0.832618,0.8557323,0.8129197,0.8102241,0.8360378,0.8528654,0.7496274,0.8586239,0.827748,0.8143048,0.8440703,0.8672183,0.8380041,0.8750833,0.8170009,0.7350944,0.8591328
cg00002426,0.7645161,0.7158301,0.8224734,0.7941364,0.7883087,0.829765,0.8144186,0.8180991,0.7498341,0.7288609,0.7437223,0.7602639,0.7754078,0.8109552,0.8236727,0.8055925,0.7740793,0.779402,0.7778447,0.6832765,0.7757921,0.8471154,0.8368915,0.7676259,0.8153682,0.7935035,0.8331429,0.7884498,0.7873183,0.8205128,0.8007419,0.7872341,0.8386483,0.8348008,0.8238532,0.7807592,0.7657841,0.797676,0.7736298,0.7727273,0.7151849,0.7946928,0.8215266,0.8342375,0.826178,0.8186302,0.8634051,0.6944206,0.8698558,0.75811,...,0.7101293,0.7480226,0.7448559,0.7468085,0.7766234,0.7827381,0.7402827,0.8239228,0.767364,0.7834483,0.8,0.7642857,0.7556237,0.7146433,0.7970732,0.6801427,0.7187808,0.7073171,0.6094069,0.7001128,0.6957123,0.6260032,0.5515587,0.7785433,0.784396,0.6100841,0.7280488,0.8239896,0.7928389,0.768797,0.7778846,0.8058902,0.7346514,0.8186453,0.8169717,0.7895414,0.8328004,0.880655,0.8391249,0.8446327,0.8592364,0.851864,0.809823,0.8461199,0.8265583,0.7975575,0.7604938,0.9015692,0.7930062,0.848166
cg00003994,0.2880215,0.2041932,0.2,0.1823755,0.1812245,0.1212121,0.1278882,0.1542253,0.2297078,0.1818996,0.1806209,0.1902874,0.1864275,0.1698473,0.193007,0.1398437,0.170339,0.2266527,0.2223169,0.1436123,0.2105263,0.2053291,0.1552483,0.1494141,0.1083004,0.2109557,0.1246032,0.2117963,0.1909354,0.1934959,0.148686,0.1628866,0.1556728,0.1624088,0.1734213,0.1739447,0.2681992,0.1499203,0.2151482,0.150289,0.2954048,0.2050833,0.2433735,0.2405063,0.1376518,0.2029851,0.1619355,0.2362288,0.2207715,0.1792693,...,0.2071429,0.189781,0.2837209,0.2174419,0.1875,0.2171779,0.1274725,0.1601273,0.1181102,0.138245,0.1340524,0.1809524,0.1576227,0.1996616,0.2296296,0.1689441,0.1878225,0.1845238,0.2698795,0.2238806,0.1551205,0.2318841,0.2583333,0.1720818,0.1708861,0.2659381,0.1824324,0.1437909,0.1410382,0.2205683,0.1767554,0.1268908,0.1272345,0.1717724,0.1184211,0.09774882,0.1131742,0.07986447,0.08927424

In [178]:
tissuedf

,tissue_src
0,"""genomic DNA from whole blood DDC117"""
1,"""genomic DNA from whole blood DN331"""
2,"""genomic DNA from whole blood DDC129"""
3,"""genomic DNA from whole blood DC478"""
4,"""genomic DNA from whole blood DC147"""
5,"""genomic DNA from whole blood DN120"""
6,"""genomic DNA from whole blood DN178"""
7,"""genomic DNA from whole blood DC133"""
8,"""genomic DNA from whole blood DN199"""
9,"""genomic DNA from whole blood DN150"""


In [ ]:
#ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE38nnn/GSE38291/suppl/GSE38291_signal_intensities.txt.gz
from urllib2 import urlopen
import ftplib

def ftp_listfiles(ftplink):
    files = []
    ftp = ftplib.FTP("www.python.org")
    try:
        files = ftp.nlst()
    except ftplib.error_perm, resp:
        if str(resp) == "550 No files found":
            print "No files in this directory"
        else:
            raise
    for f in files:
        print f


def fetch_signalfiles(geo_noncancer_list):
    gse_noncancer_edge=[]
    failed_fetch_list = []
    for i in geo_noncancer_list:
        key_num = i.split("GSE")[1]
        url_key= "GSE"+key_num[0:2]
        file_n = i + "_series_matrix.txt.gz"
        url = "ftp://ftp.ncbi.nlm.nih.gov/geo/series/"+url_key.replace(" ","")+ "nnn/" + i + "/suppl/"
        print url
        ftp_listfiles(url)
        ftplink= (url + file_n).replace(" ","")
        if result == 1:
            gse_noncancer_edge.append(i)
        else:
            print i, " success"
            '''pfetch = Popen("ftp " + ftplink,stdout=PIPE,stderr=PIPE,shell=True,close_fds=True)
            std_out, std_err = pfetch.communicate()
            exit_code = pfetch.returncode
            if exit_code:
                failed_fetch_list.append(i)'''
            #os.system("ftp " + ftplink)
    return gse_noncancer_edge, failed_fetch_list    

In [ ]:
fetch_signalfiles(geo_noncancer_list)